In [1]:
import ee
import folium
import io
import os
import time
from PIL import Image
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

ee.Initialize()

# foliumマップにGEEを表示させる関数
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict["tile_fetcher"].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True,
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer


In [2]:
# ポリゴン座標（Research4_chiba_detail.ipynbと同じ）
coords = [
    [140.048947, 35.14381],
    [140.048947, 35.161424],
    [140.068946, 35.161424],
    [140.068946, 35.14381],
    [140.048947, 35.14381]
]
stat_region = ee.Geometry.Polygon(coords)

# Sentinel-1のデータを取得（2025年9月1日～11月7日）
start_date = "2025-09-01"
end_date = "2025-11-07"

# Sentinel-1 GRDデータセットを取得
sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD")

# 期間とポリゴンでフィルタリング
s1_collection = (
    sentinel1
    .filterDate(f"{start_date}T00:00:00", f"{end_date}T23:59:59")
    .filterBounds(stat_region)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))  # または 'DESCENDING'
    .sort("system:time_start")
)

# コレクションのサイズを確認
collection_size = s1_collection.size().getInfo()
print(f"✓ Sentinel-1画像を{collection_size}枚取得しました（期間: {start_date} ～ {end_date}）")

# 最初と最後の画像の情報を表示
if collection_size > 0:
    first_image = s1_collection.first()
    last_image = s1_collection.sort("system:time_start", False).first()
    
    first_date = ee.Date(first_image.get("system:time_start")).format("YYYY-MM-dd").getInfo()
    last_date = ee.Date(last_image.get("system:time_start")).format("YYYY-MM-dd").getInfo()
    
    print(f"  最初の画像: {first_date}")
    print(f"  最後の画像: {last_date}")
    
    # 画像のバンド情報を表示
    bands = first_image.bandNames().getInfo()
    print(f"  利用可能なバンド: {bands}")
    
    # コレクションを保存（後で使用するため）
    s1_images = s1_collection
else:
    print("⚠ 画像が見つかりませんでした")
    s1_images = None


✓ Sentinel-1画像を4枚取得しました（期間: 2025-09-01 ～ 2025-11-07）
  最初の画像: 2025-09-22
  最後の画像: 2025-10-28
  利用可能なバンド: ['VV', 'VH', 'angle']


In [5]:
# Sentinel-1画像をポリゴン内で表示

if s1_images and collection_size > 0:
    # コレクションをメディアン合成（期間全体の代表画像）
    s1_median = s1_images.median()
    
    # VVバンドを選択してポリゴン内でクリップ
    s1_vv = s1_median.select('VV').clip(stat_region)
    
    # マップの中心点を計算
    center_lat = (35.14381 + 35.161424) / 2
    center_lon = (140.048947 + 140.068946) / 2
    
    # Foliumマップを作成
    s1_map = folium.Map(location=[center_lat, center_lon], zoom_start=14)
    
    # Sentinel-1の可視化パラメータ（VVバンド）
    # デシベル値の範囲を設定（通常-25から5 dB）
    visualization_s1 = {
        "min": -25,
        "max": 5,
        "palette": ["black", "white"]
    }
    
    # Sentinel-1画像を追加
    s1_map.add_ee_layer(s1_vv, visualization_s1, "Sentinel-1 VV")
    folium.Map.add_ee_layer = add_ee_layer
    
    # ポリゴンの境界線を追加
    polygon_coords_folium = [[coord[1], coord[0]] for coord in coords[:-1]]
    folium.Polygon(
        locations=polygon_coords_folium,
        color='blue',
        weight=3,
        fillColor='blue',
        fillOpacity=0.1,
        popup='Research Area (Polygon)',
        tooltip='Research Area'
    ).add_to(s1_map)
    
    # マップの範囲を設定
    bounds = [
        [min(coord[1] for coord in coords[:-1]), min(coord[0] for coord in coords[:-1])],
        [max(coord[1] for coord in coords[:-1]), max(coord[0] for coord in coords[:-1])]
    ]
    s1_map.fit_bounds(bounds, padding=(10, 10))
    
    # レイヤーコントロールを追加
    s1_map.add_child(folium.LayerControl(collapsed=False))
    
    # マップを表示
    print(f"\n✓ Sentinel-1画像をマップに表示しました")
    print(f"  期間: {start_date} ～ {end_date}")
    print(f"  バンド: VV")
    print(f"  表示範囲: ポリゴン内")
    
    s1_map
else:
    print("⚠ Sentinel-1画像が取得できていないため、マップを表示できません")



✓ Sentinel-1画像をマップに表示しました
  期間: 2025-09-01 ～ 2025-11-07
  バンド: VV
  表示範囲: ポリゴン内


In [ ]:
# 日付ごとにSentinel-1画像のマップを取得

if s1_images and collection_size > 0:
    # 各画像の日付を取得
    image_list = s1_images.toList(collection_size)
    
    # マップの中心点を計算
    center_lat = (35.14381 + 35.161424) / 2
    center_lon = (140.048947 + 140.068946) / 2
    
    # ポリゴンの境界線座標
    polygon_coords_folium = [[coord[1], coord[0]] for coord in coords[:-1]]
    bounds = [
        [min(coord[1] for coord in coords[:-1]), min(coord[0] for coord in coords[:-1])],
        [max(coord[1] for coord in coords[:-1]), max(coord[0] for coord in coords[:-1])]
    ]
    
    # 各画像のマップを作成
    s1_maps = {}
    
    for i in range(collection_size):
        image = ee.Image(image_list.get(i))
        
        # 画像の日付を取得
        image_date = ee.Date(image.get("system:time_start")).format("YYYY-MM-dd").getInfo()
        
        # VVバンドを選択してポリゴン内でクリップ
        s1_vv = image.select('VV').clip(stat_region)
        
        # Foliumマップを作成
        s1_map = folium.Map(location=[center_lat, center_lon], zoom_start=14)
        
        # Sentinel-1の可視化パラメータ
        visualization_s1 = {
            "min": -25,
            "max": 5,
            "palette": ["black", "white"]
        }
        
        # Sentinel-1画像を追加
        s1_map.add_ee_layer(s1_vv, visualization_s1, f"Sentinel-1 VV ({image_date})")
        folium.Map.add_ee_layer = add_ee_layer
        
        # ポリゴンの境界線を追加
        folium.Polygon(
            locations=polygon_coords_folium,
            color='blue',
            weight=3,
            fillColor='blue',
            fillOpacity=0.1,
            popup=f'Research Area (Polygon) - {image_date}',
            tooltip='Research Area'
        ).add_to(s1_map)
        
        # マップの範囲を設定
        s1_map.fit_bounds(bounds, padding=(10, 10))
        
        # レイヤーコントロールを追加
        s1_map.add_child(folium.LayerControl(collapsed=False))
        
        # マップを保存
        s1_maps[image_date] = s1_map
        
        print(f"✓ {image_date}のマップを作成しました")
    
    print(f"\n✓ 全{len(s1_maps)}枚のマップを作成しました")
    print(f"  日付: {list(s1_maps.keys())}")
    
    # 最初のマップを表示
    if s1_maps:
        first_date = list(s1_maps.keys())[0]
        print(f"\n最初のマップ（{first_date}）を表示します:")
        s1_maps[first_date]
    
else:
    print("⚠ Sentinel-1画像が取得できていないため、マップを表示できません")


✓ 2025-09-22のレイヤーを追加しました
✓ 2025-10-04のレイヤーを追加しました
✓ 2025-10-16のレイヤーを追加しました
✓ 2025-10-28のレイヤーを追加しました

✓ 1つのマップに全4枚のレイヤーを追加しました
  日付: ['2025-09-22', '2025-10-04', '2025-10-16', '2025-10-28']
  マップ右上のレイヤーコントロールで日付を選択できます

✓ マップを 'sentinel1_interactive_map.html' に保存しました
  ブラウザで開いて、レイヤーコントロールから日付を選択できます


In [18]:
# 1つのマップ（日付選択可能）をHTMLファイルとして保存

# セル3で作成したマップを保存
try:
    if 's1_map' in globals() and s1_map is not None:
        output_file = "sentinel1_interactive_map.html"
        output_path = os.path.join(os.getcwd(), output_file)
        s1_map.save(output_file)
        print(f"✓ マップを '{output_file}' に保存しました")
        print(f"  保存先: {output_path}")
        print(f"  ブラウザで開いて、レイヤーコントロールから日付を選択できます")
        if 'dates_list' in globals():
            print(f"  保存された日付: {sorted(dates_list)}")
    else:
        print("⚠ マップが作成されていません。先にセル3を実行してください。")
        print("  セル3を実行してから、このセルを再実行してください。")
except Exception as e:
    print(f"⚠ エラーが発生しました: {str(e)}")
    print("  セル3を実行してから、このセルを再実行してください。")


✓ マップを 'sentinel1_interactive_map.html' に保存しました
  保存先: /Users/seimiyaakirayuu/Desktop/project/geo_practice/1_satelite_practice/sentinel1_interactive_map.html
  ブラウザで開いて、レイヤーコントロールから日付を選択できます
  保存された日付: ['2025-09-22', '2025-10-04', '2025-10-16', '2025-10-28']


In [ ]:
# RVI (Radar Vegetation Index) を計算して1つのマップで日付を選択できるように表示・保存
# RVI = (4 * VH) / (VV + VH)

if s1_images and collection_size > 0:
    # 各画像の日付を取得
    image_list = s1_images.toList(collection_size)
    
    # マップの中心点を計算
    center_lat = (35.14381 + 35.161424) / 2
    center_lon = (140.048947 + 140.068946) / 2
    
    # ポリゴンの境界線座標
    polygon_coords_folium = [[coord[1], coord[0]] for coord in coords[:-1]]
    bounds = [
        [min(coord[1] for coord in coords[:-1]), min(coord[0] for coord in coords[:-1])],
        [max(coord[1] for coord in coords[:-1]), max(coord[0] for coord in coords[:-1])]
    ]
    
    # RVI計算関数
    def calculate_rvi(image):
        """RVI = (4 * VH) / (VV + VH) を計算"""
        vv = image.select('VV')
        vh = image.select('VH')
        rvi = vh.multiply(4).divide(vv.add(vh)).rename('RVI')
        return image.addBands(rvi)
    
    # 全画像のRVI統計情報を取得（可視化範囲の統一設定用）
    print("RVIを計算中...")
    all_rvi_values = []
    dates_list = []
    
    for i in range(collection_size):
        image = ee.Image(image_list.get(i))
        image_date = ee.Date(image.get("system:time_start")).format("YYYY-MM-dd").getInfo()
        dates_list.append(image_date)
        
        image_with_rvi = calculate_rvi(image)
        rvi_band = image_with_rvi.select('RVI').clip(stat_region)
        
        rvi_stats = rvi_band.reduceRegion(
            reducer=ee.Reducer.minMax().combine(
                ee.Reducer.mean().combine(ee.Reducer.stdDev(), '', True), '', True
            ),
            geometry=stat_region,
            scale=10,
            maxPixels=1e9,
            bestEffort=True
        ).getInfo()
        
        all_rvi_values.append({
            'min': rvi_stats.get('RVI_min', 0),
            'max': rvi_stats.get('RVI_max', 1),
            'mean': rvi_stats.get('RVI_mean', 0.5),
            'std': rvi_stats.get('RVI_stdDev', 0.1)
        })
        print(f"  {image_date}: RVI範囲 [{all_rvi_values[-1]['min']:.4f}, {all_rvi_values[-1]['max']:.4f}], 平均: {all_rvi_values[-1]['mean']:.4f}")
    
    # 全画像のRVI統計情報から可視化範囲を決定
    global_rvi_min = min([v['min'] for v in all_rvi_values])
    global_rvi_max = max([v['max'] for v in all_rvi_values])
    global_rvi_mean = sum([v['mean'] for v in all_rvi_values]) / len(all_rvi_values)
    global_rvi_std = sum([v['std'] for v in all_rvi_values]) / len(all_rvi_values)
    
    print(f"\n全画像のRVI統計: 最小={global_rvi_min:.4f}, 最大={global_rvi_max:.4f}, 平均={global_rvi_mean:.4f}, 標準偏差={global_rvi_std:.4f}")
    
    # 1つのマップを作成
    rvi_map = folium.Map(location=[center_lat, center_lon], zoom_start=14)
    
    # RVIの可視化パラメータ（全画像で統一、グレースケール）
    # 実際のRVI値の範囲に基づいて設定（外れ値を除外）
    vis_min = max(global_rvi_min, global_rvi_mean - 3 * global_rvi_std)
    vis_max = min(global_rvi_max, global_rvi_mean + 3 * global_rvi_std)
    
    print(f"可視化範囲: [{vis_min:.4f}, {vis_max:.4f}]")
    
    visualization_rvi = {
        "min": vis_min,
        "max": vis_max,
        "palette": ["black", "white"]  # グレースケール
    }
    
    # 各日付のRVIをレイヤーとして追加
    for i in range(collection_size):
        image = ee.Image(image_list.get(i))
        image_date = dates_list[i]
        
        # RVIを計算
        image_with_rvi = calculate_rvi(image)
        rvi_band = image_with_rvi.select('RVI').clip(stat_region)
        
        # 各日付のRVIをレイヤーとして追加
        rvi_map.add_ee_layer(rvi_band, visualization_rvi, f"RVI ({image_date})")
        folium.Map.add_ee_layer = add_ee_layer
        
        print(f"✓ {image_date}のRVIレイヤーを追加しました")
    
    # 最後の日のRGB画像を追加
    last_image = ee.Image(image_list.get(collection_size - 1))
    last_date = dates_list[-1]
    
    # RGB合成: Red=VV, Green=VH, Blue=VV/VHの比率
    # バンドをリネームしてRGB合成を作成
    vv_band = last_image.select('VV').rename('R')
    vh_band = last_image.select('VH').rename('G')
    # BlueバンドはVV/VHの比率を使用（または単純にVVを使用）
    blue_band = last_image.select('VV').divide(last_image.select('VH').add(0.001)).rename('B')  # 0.001を追加してゼロ除算を回避
    
    rgb_image = ee.Image.cat([vv_band, vh_band, blue_band]).clip(stat_region)
    
    # RGB可視化パラメータ（デシベル値を0-255にスケール）
    # 各バンドの範囲を設定
    visualization_rgb = {
        "min": -25,
        "max": 5,
        "bands": ["R", "G", "B"]
    }
    
    rvi_map.add_ee_layer(rgb_image, visualization_rgb, f"RGB ({last_date})")
    folium.Map.add_ee_layer = add_ee_layer
    
    print(f"✓ {last_date}のRGBレイヤーを追加しました")
    
    # ポリゴンの境界線を追加
    folium.Polygon(
        locations=polygon_coords_folium,
        color='blue',
        weight=3,
        fillColor='blue',
        fillOpacity=0.1,
        popup='Research Area (Polygon)',
        tooltip='Research Area'
    ).add_to(rvi_map)
    
    # マップの範囲を設定
    rvi_map.fit_bounds(bounds, padding=(10, 10))
    
    # レイヤーコントロールを追加（レイヤーの表示/非表示を切り替え可能）
    rvi_map.add_child(folium.LayerControl(collapsed=False))
    
    print(f"\n✓ 1つのマップに全{len(dates_list)}枚のRVIレイヤーを追加しました")
    print(f"  日付: {sorted(dates_list)}")
    print(f"  マップ右上のレイヤーコントロールで日付を選択できます")
    
    # HTMLファイルとして保存
    output_file = "rvi_interactive_map.html"
    rvi_map.save(output_file)
    print(f"\n✓ RVIマップを '{output_file}' に保存しました")
    print(f"  ブラウザで開いて、レイヤーコントロールから日付を選択できます")
    
    # マップを表示
    rvi_map
    
else:
    print("⚠ Sentinel-1画像が取得できていないため、RVIを計算できません")


RVIを計算中...
  2025-09-22: RVI範囲 [-319.0183, 504.1020], 平均: 2.5529
  2025-10-04: RVI範囲 [-442.4110, 268.4236], 平均: 2.5851
  2025-10-16: RVI範囲 [-379.6363, 134.2666], 平均: 2.5548
  2025-10-28: RVI範囲 [-70.0537, 563.9151], 平均: 2.6271

全画像のRVI統計: 最小=-442.4110, 最大=563.9151, 平均=2.5800, 標準偏差=3.4868
可視化範囲: [-7.8803, 13.0402]
✓ 2025-09-22のRVIレイヤーを追加しました
✓ 2025-10-04のRVIレイヤーを追加しました
✓ 2025-10-16のRVIレイヤーを追加しました
✓ 2025-10-28のRVIレイヤーを追加しました


EEException: Image.select: Can't add a band named 'VV' to image because a band with this name already exists. Existing bands: [VV, VH].

RVIを計算中...
  2025-09-22: RVI範囲 [-319.0183, 504.1020], 平均: 2.5529
  2025-10-04: RVI範囲 [-442.4110, 268.4236], 平均: 2.5851
  2025-10-16: RVI範囲 [-379.6363, 134.2666], 平均: 2.5548
  2025-10-28: RVI範囲 [-70.0537, 563.9151], 平均: 2.6271

全画像のRVI統計: 最小=-442.4110, 最大=563.9151, 平均=2.5800
✓ 2025-09-22のRVIレイヤーを追加しました
✓ 2025-10-04のRVIレイヤーを追加しました
✓ 2025-10-16のRVIレイヤーを追加しました
✓ 2025-10-28のRVIレイヤーを追加しました

✓ 1つのマップに全4枚のRVIレイヤーを追加しました
  日付: ['2025-09-22', '2025-10-04', '2025-10-16', '2025-10-28']
  マップ右上のレイヤーコントロールで日付を選択できます

✓ RVIマップを 'rvi_interactive_map.html' に保存しました
  ブラウザで開いて、レイヤーコントロールから日付を選択できます
